# Análisis Exploratorio de Datos - FONASA

Este notebook tiene como objetivo explorar una base de datos de *FONASA* en formato `.csv`, identificar variables relevantes para el estudio de **migración y salud**, y generar un dataset para análisis posteriores.

## Importar librerias
Inicialmente importamos las librerias necesarias para el funcionamiento del notebook.

In [1]:
# Librerías necesarias
import pandas as pd

## Lectura de archivo: FONASA
Se lee el archivo utilizando pandas, por el tipo de archivo se debe especificar la encriptacion a *latin1*.<br>
El archivo se obtuvo desde datos abiertos de *FONASA*, se puede obtener directamente en el siguiente [enlace](https://public.tableau.com/views/Beneficiarios_16704273822880/Reporte??:showVizHome=no), seccion *catálogo de datos*.

In [15]:
# Ruta al archivo 
ruta = '../../data/raw_data/fonasa_2024.csv'

# Leer el CSV
df = pd.read_csv(ruta,encoding='latin1')

# Ver las primeras filas
df.head()

,MES_INFORMACION,REGIÓN,COMUNA,SEXO,EDAD_TRAMO,NACIONALIDAD,TITULAR_CARGA,TRAMO,BENEFICIARIOS
0,202412,Metropolitana De Santiago,Macul,Mujer,50 a 54 años,Chilena,Titular,C,570
1,202412,Metropolitana De Santiago,Macul,Hombre,30 a 34 años,Chilena,Titular,D,1810
2,202412,Del Bíobío,Penco,Mujer,45 a 49 años,Chilena,Titular,B,518
3,202412,Del Bíobío,Antuco,Hombre,65 a 69 años,Chilena,Titular,B,68
4,202412,De La Araucanía,Temuco,Mujer,25 a 29 años,Chilena,Titular,D,2414


### Limpieza de archivo
Inicialmente debemos limpiar los archivos de valores nulos

In [16]:
df[df["COMUNA"] == "Desconocida"].head()

,MES_INFORMACION,REGIÓN,COMUNA,SEXO,EDAD_TRAMO,NACIONALIDAD,TITULAR_CARGA,TRAMO,BENEFICIARIOS
94,202412,Desconocida,Desconocida,Hombre,S.I.,Chilena,Titular,B,3057
95,202412,Desconocida,Desconocida,Hombre,25 a 29 años,Chilena,Titular,D,2700
443,202412,Desconocida,Desconocida,Hombre,25 a 29 años,Extranjera,Titular,A,234
485,202412,Desconocida,Desconocida,Mujer,00 a 4 años,Chilena,Titular,A,3289
1049,202412,Desconocida,Desconocida,Hombre,50 a 54 años,Chilena,Titular,C,1041


In [17]:
# Filtramos para eliminar valores nulos
df = df[df["COMUNA"] != "Desconocida"]

### Agrupaciones
Por el formato del dataset debemos agrupar por columnas, con eso podemos obtener el total de beneficiarios por región y comuna, especificando si son de nacionalidad Chilena o extranjera.

In [18]:
df_grouped = (
    df.groupby(["REGIÓN", "COMUNA", "NACIONALIDAD"])["BENEFICIARIOS"]
      .sum()
      .reset_index()
)
df_grouped.head(10)

,REGIÓN,COMUNA,NACIONALIDAD,BENEFICIARIOS
0,De Antofagasta,Antofagasta,Chilena,263334
1,De Antofagasta,Antofagasta,Extranjera,86784
2,De Antofagasta,Calama,Chilena,112470
3,De Antofagasta,Calama,Extranjera,41553
4,De Antofagasta,María Elena,Chilena,1915
5,De Antofagasta,María Elena,Extranjera,739
6,De Antofagasta,Mejillones,Chilena,4917
7,De Antofagasta,Mejillones,Extranjera,4179
8,De Antofagasta,Ollagüe,Chilena,32
9,De Antofagasta,Ollagüe,Extranjera,33


### Pivotar la nacionalidad a columnas
Debemos mover los datos correspondiente a tipo de nacionalidad a columnas para facilitar calculos posteriores, esto permitira que existan dos columnas especificando a el total de beneficiaros por nacionalidad.

In [19]:
# Pivotar nacionalidad a columnas
df_pivot = df_grouped.pivot_table(
    index=["REGIÓN", "COMUNA"],
    columns="NACIONALIDAD",
    values="BENEFICIARIOS",
    aggfunc="sum",
    fill_value=0
).reset_index()

# Renombrar columnas para que queden más claras
df_pivot = df_pivot.rename(columns={
    "Chilena": "BENEFICIARIOS_CHILENOS",
    "Extranjera": "BENEFICIARIOS_EXTRANJEROS"
})

df_pivot.columns.name = None
df_pivot.head(10)

,REGIÓN,COMUNA,BENEFICIARIOS_CHILENOS,BENEFICIARIOS_EXTRANJEROS
0,De Antofagasta,Antofagasta,263334,86784
1,De Antofagasta,Calama,112470,41553
2,De Antofagasta,María Elena,1915,739
3,De Antofagasta,Mejillones,4917,4179
4,De Antofagasta,Ollagüe,32,33
5,De Antofagasta,San Pedro De Atacama,8787,3845
6,De Antofagasta,Sierra Gorda,385,532
7,De Antofagasta,Taltal,6158,913
8,De Antofagasta,Tocopilla,14630,2542
9,De Arica Y Parinacota,Arica,193880,47089


### Creacion de nuevas columnas
Se crean nuevas columnas correspondiente al calculo del total de beneficiarios por comuna y a su vez esto permite calcular el porcentaje correspondiente a beneficiarios extranjero del total de los beneficiarios.

In [20]:
# Calcular total de beneficiarios
df_pivot["TOTAL_BENEFICIARIOS"] = (
    df_pivot["BENEFICIARIOS_CHILENOS"] + df_pivot["BENEFICIARIOS_EXTRANJEROS"]
)

# Calcular porcentaje de extranjeros
df_pivot["PORCENTAJE_EXTRANJEROS"] = (
    df_pivot["BENEFICIARIOS_EXTRANJEROS"] / df_pivot["TOTAL_BENEFICIARIOS"] * 100
)

# redondear a 2 decimales
df_pivot["PORCENTAJE_EXTRANJEROS"] = df_pivot["PORCENTAJE_EXTRANJEROS"].round(2)

df_pivot.head()

,REGIÓN,COMUNA,BENEFICIARIOS_CHILENOS,BENEFICIARIOS_EXTRANJEROS,TOTAL_BENEFICIARIOS,PORCENTAJE_EXTRANJEROS
0,De Antofagasta,Antofagasta,263334,86784,350118,24.79
1,De Antofagasta,Calama,112470,41553,154023,26.98
2,De Antofagasta,María Elena,1915,739,2654,27.84
3,De Antofagasta,Mejillones,4917,4179,9096,45.94
4,De Antofagasta,Ollagüe,32,33,65,50.77


## Lectura de archivo: Codigo de comuna
Se debe leer el archivo correspondiente al codigo de las comunas, esto para agregarlo al dataset principal, lo que permitira facilitar los cruces de datos con otras tablas al no depender del nombre de la comuna y si del codigo de esta.

In [21]:
ruta_comunas = '../../data/processed_data/codigo_comuna.csv'

df_comunas =  pd.read_csv(ruta_comunas)

df_comunas.head()

,Código comuna,Comuna
0,15101,Arica
1,15102,Camarones
2,15201,Putre
3,15202,General Lagos
4,1101,Iquique


## Verificacion nombres de comunas
Se debe verificar que los nombres de las comunas corresponda a los nombres en el archivo con el codigo comunal, esto permitira hacer la union entre ambos datasets.

In [22]:
# Comunas en minuscula para comparar
df_comunas['Comuna'] = df_comunas['Comuna'].str.lower()
df_pivot['COMUNA'] = df_pivot['COMUNA'].str.lower()

# Comunas faltantes en df_comunas
comunas_faltantes_comunas = set(df_pivot["COMUNA"]) - set(df_comunas["Comuna"])

# Comunas faltantes en df_pivot
comunas_faltantes_df = set(df_comunas["Comuna"]) - set(df_pivot["COMUNA"])

print("Comunas faltantes en df_pivot:")
print(comunas_faltantes_df)

print("\nComunas faltantes en df_comunas:")
print(comunas_faltantes_comunas)

Comunas faltantes en df_pivot:
{'trehuaco', 'ránquil', 'cabo de hornos'}

Comunas faltantes en df_comunas:
{'treguaco', 'ranquil', 'cabo de hornos (ex - navarino)'}


In [23]:
# Reemplazamos los nombres de las comunas para que coincidan con las de codigo comunal
df_pivot["COMUNA"] = df_pivot["COMUNA"].replace({
    "treguaco": "trehuaco",
    "cabo de hornos (ex - navarino)": "cabo de hornos",
    "ranquil":"ránquil"
})

## Union de Datasets y exportacion a csv
Una vez comprobado que las comunas de ambos datasets coinciden, se unen en un dataset final el cual contiene los datos que se almacenaran en el dataset final.<br>
Luego se exportara a formato csv para ser utilizado en el estudio.

In [24]:
# Join en comunas
df_final = df_comunas.merge(df_pivot, left_on="Comuna", right_on="COMUNA", how="inner")

# Se elimina columna repetida
df_final.drop(columns=['COMUNA'], inplace=True)

df_final.head()

,Código comuna,Comuna,REGIÓN,BENEFICIARIOS_CHILENOS,BENEFICIARIOS_EXTRANJEROS,TOTAL_BENEFICIARIOS,PORCENTAJE_EXTRANJEROS
0,15101,arica,De Arica Y Parinacota,193880,47089,240969,19.54
1,15102,camarones,De Arica Y Parinacota,68,131,199,65.83
2,15201,putre,De Arica Y Parinacota,319,304,623,48.80
3,15202,general lagos,De Arica Y Parinacota,102,151,253,59.68
4,1101,iquique,De Tarapacá,146848,54352,201200,27.01


In [25]:
# Se exporta a csv
df_final.to_csv('../../data/processed_data/fonasa.csv', index=False)